In [ ]:
%cd ..

In [ ]:
import ibkr
import schwab
from analysis import liveValuesForPositions, deduplicatePositions
from decimal import Decimal
from ib_insync import IB
from itertools import chain
from model import Cash, Currency, Stock
from pathlib import Path
import pandas as pd

In [ ]:
# The currency in which to consider portfolio balancing. Positions denominated in other currencies will be ignored.
# TODO: Support multi-currency portfolios
PORTFOLIO_CURRENCY = Currency.USD

# Desired % of portfolio to allocate to each stock in the list.
DESIRED_ALLOCATION = {
    'TSLA': 0.02,
    'AAPL': 0.05,
    'BRK B': 0.05,
    'VT': 0.8,
    'BND': 0.08,
}

In [ ]:
totalAllocation = sum(DESIRED_ALLOCATION.values())
assert abs(totalAllocation - 1) < 0.0001, f'Allocations do not total 100%, got {totalAllocation:.2%}'

In [ ]:
ib = IB()
ib.connect('127.0.0.1', port = 4001)
positions = ibkr.downloadPositions(ib, lenient=False)

In [ ]:
path = Path('mydata/schwab-positions.CSV')
if path.exists():
    positions = deduplicatePositions(chain(positions, schwab.parsePositions(path)))

In [ ]:
stockPositions = [p for p in positions if isinstance(p.instrument, Stock) and p.instrument.currency == PORTFOLIO_CURRENCY]
stockPositions.sort(key=lambda p: p.instrument)
stockPositions

In [ ]:
values = liveValuesForPositions(stockPositions, ibkr.IBDataProvider(ib))

In [ ]:
portfolioValue = sum((x for x in values.values()), Cash(currency = PORTFOLIO_CURRENCY, quantity = Decimal(0)))
str(portfolioValue)

In [ ]:
def color_deviations(val):
    max_deviation = 0.02
    color = 'black'
    if abs(val) > max_deviation:
        if val > 0:
            color = 'green'
        else:
            color = 'red'
            
    return f'color: {color}'

In [ ]:
rows = {p.instrument.symbol: [
    values[p],
    float(values[p].quantity) / float(portfolioValue.quantity),
    DESIRED_ALLOCATION.get(p.instrument.symbol),
    float(values[p].quantity) / float(portfolioValue.quantity) - (DESIRED_ALLOCATION.get(p.instrument.symbol, 0)),
] for p in stockPositions}

In [ ]:
df = pd.DataFrame.from_dict(data=rows, orient='index', columns=[
    'Market value',
    '% of portfolio',
    'Desired %',
    'Deviation'
])

df.style.format({
    '% of portfolio': '{:.2%}',
    'Desired %': '{:.2%}',
    'Deviation': '{:.2%}'
}).applymap(color_deviations, 'Deviation')